In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import scipy.signal as signal

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
#  TODO: Add functionality to handle joining multiple csvs together from different file locations into usable df

In [4]:
path = "/home/eric/Desktop/data/treadmill/1739140957.3850882.csv"

df = pd.read_csv(path, header=None)
df.columns = df.iloc[0]
df = df.drop(0, axis=0).reset_index(drop=True)


/tmp/ipykernel_26302/2471569208.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=None)


In [5]:
# Drop unnecessary columns
columns_to_drop = [col for col in df.columns if isinstance(col, str) and ('IMP' in col.upper() or 'TIME' in col.upper()) and col != 'time 0']
df = df.drop(columns=columns_to_drop)
df = df.drop(df.columns[0], axis=1)
df = df.apply(pd.to_numeric, errors='coerce')

In [6]:
df = df[200:390000]

In [7]:
df

,time 0,sensor 0 EMG 1 (1259.259 Hz),sensor 0 ACC X (148.148 Hz),sensor 0 ACC Y (148.148 Hz),sensor 0 ACC Z (148.148 Hz),sensor 0 GYRO X (148.148 Hz),sensor 0 GYRO Y (148.148 Hz),sensor 0 GYRO Z (148.148 Hz),sensor 1 EMG 1 (1259.259 Hz),sensor 1 ACC X (148.148 Hz),...,sensor 4 GYRO X (148.148 Hz),sensor 4 GYRO Y (148.148 Hz),sensor 4 GYRO Z (148.148 Hz),sensor 5 EMG 1 (1259.259 Hz),sensor 5 ACC X (148.148 Hz),sensor 5 ACC Y (148.148 Hz),sensor 5 ACC Z (148.148 Hz),sensor 5 GYRO X (148.148 Hz),sensor 5 GYRO Y (148.148 Hz),sensor 5 GYRO Z (148.148 Hz)
200,1.739141e+09,-0.029038,-0.080383,-0.931030,-0.375549,11.778625,-4.053435,2.732824,-0.018967,0.244263,...,-16.572519,-1.175573,-4.091603,-0.030549,-0.139343,-0.988159,-0.031250,-12.076336,8.656488,3.198473
201,1.739141e+09,-0.027695,-0.075867,-0.931702,-0.380798,11.877863,-5.564886,2.511450,-0.019303,0.248901,...,-16.854961,-2.000000,-4.061069,-0.032395,-0.138733,-0.986511,-0.032898,-11.969465,7.641222,2.885496
202,1.739141e+09,-0.030045,-0.076904,-0.932861,-0.384583,11.167939,-6.709924,2.374046,-0.018967,0.251892,...,-17.229008,-2.274809,-4.267176,-0.033738,-0.138855,-0.983521,-0.033203,-12.022901,6.671756,2.450382
203,1.739141e+09,-0.032395,-0.078369,-0.934692,-0.387939,10.427481,-7.534351,2.137405,-0.020813,0.258179,...,-17.519083,-2.465649,-4.610687,-0.035080,-0.138672,-0.982300,-0.033325,-12.343512,5.694656,2.267176
204,1.739141e+09,-0.032731,-0.072144,-0.943176,-0.387207,10.251908,-8.267176,1.725191,-0.019638,0.261169,...,-17.702290,-2.183206,-4.618320,-0.033738,-0.139160,-0.982178,-0.036499,-12.305344,4.381680,2.114504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259959,NaN,-0.029709,NaN,NaN,NaN,NaN,NaN,NaN,-0.013931,NaN,...,NaN,NaN,NaN,-0.042130,NaN,NaN,NaN,NaN,NaN,NaN
259960,NaN,-0.027695,NaN,NaN,NaN,NaN,NaN,NaN,-0.012757,NaN,...,NaN,NaN,NaN,-0.037598,NaN,NaN,NaN,NaN,NaN,NaN
259961,NaN,-0.026184,NaN,NaN,NaN,NaN,NaN,NaN,-0.012924,NaN,...,NaN,NaN,NaN,-0.031388,NaN,NaN,NaN,NaN,NaN,NaN
259962,NaN,-0.030213,NaN,NaN,NaN,NaN,NaN,NaN,-0.011246,NaN,...,NaN,NaN,NaN,-0.028702,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
emg_columns = ['sensor 0 EMG 1 (1259.259 Hz)',
               'sensor 1 EMG 1 (1259.259 Hz)',
               'sensor 2 EMG 1 (1259.259 Hz)',
               'sensor 3 EMG 1 (1259.259 Hz)',
               'sensor 4 EMG 1 (1259.259 Hz)',
               'sensor 5 EMG 1 (1259.259 Hz)']

In [11]:
def resample_data(df, target_fs):
    """
    Resample data to the target sampling frequency.
    
    Parameters:
    - df: The DataFrame containing the time and signal data (e.g., 'time 0', EMG, GYRO, ACC columns).
    - target_fs: The target sampling frequency (Hz).
    
    Returns:
    - df_resampled: A new DataFrame with all columns resampled to the target frequency.
    """
    time_column = df['time 0'].dropna()
    if len(time_column) < 2:
        raise ValueError("Time column 'time 0' has fewer than 2 valid entries after removing NaN.")
    
    #  Calculate total duration and original sampling frequency
    total_duration = time_column.iloc[-1] - time_column.iloc[0]  # Total time span
    if np.isnan(total_duration):
        raise ValueError("Total duration is NaN. Check 'time 0' for invalid values.")
    
    original_length = len(time_column)  #  Number of valid samples in time column
    original_fs = original_length / total_duration  #  Average original sampling frequency (Hz)
    
    #  Calculate target length based on target_fs
    target_length = int(total_duration * target_fs)  #  Number of samples at target frequency
    if target_length <= 0:
        raise ValueError(f"Target length {target_length} is invalid. Check total_duration and target_fs.")
    
    new_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], target_length)  # New time array
    
    df_resampled = pd.DataFrame({'time 0': new_time})
    
    emg_columns = [col for col in df.columns if 'EMG' in col]
    gyro_accel_columns = [col for col in df.columns if 'GYRO' in col or 'ACC' in col]
    
    #  Resample all signal columns to the target length
    for col in emg_columns + gyro_accel_columns:
        original_signal = df[col].dropna()
        if len(original_signal) < 2:
            print(f"Warning: Column '{col}' has fewer than 2 valid entries. Filling with zeros.")
            df_resampled[col] = np.zeros(target_length)
            continue
        
        #  Interpolate to match time column length if needed
        if len(original_signal) != original_length:
            original_time = np.linspace(time_column.iloc[0], time_column.iloc[-1], len(original_signal))
            interpolated_signal = np.interp(time_column, original_time, original_signal)
            resampled_signal = signal.resample(interpolated_signal, target_length)
        else:
            resampled_signal = signal.resample(original_signal, target_length)
        df_resampled[col] = resampled_signal
    
    return df_resampled

In [12]:
#  Implement butterworth filters of different orders, cutoff frequencies, sampling frequencies, and filter_band type to columns of a dataframe.

def butterworth_filter(data, order, cutoff, fs, filter_band='low'):
    nyquist = 0.5 * fs
    normalized_cutoff = np.array(cutoff) / nyquist
    
    #  Create Butterworth filter coefficients
    if filter_band == 'low':
        b, a = signal.butter(order, normalized_cutoff[0], btype='low')
    elif filter_band == 'high':
        b, a = signal.butter(order, normalized_cutoff[0], btype='high')
    elif filter_band == 'bandpass':
        b, a = signal.butter(order, normalized_cutoff, btype='bandpass')
    elif filter_band == 'bandstop':
        b, a = signal.butter(order, normalized_cutoff, btype='bandstop')
    else:
        raise ValueError(f"unsupported filter: {filter_band}")
    
    #  Apply filter
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

def apply_filter(df, columns, order=4, cutoff=[1.0], fs=1000, filter_band='low'):
    '''
    Apply a Butterworth filter of specified order and cutoff to the given columns of the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the filter to (e.g., EMG columns).
    - order: The order of the filter.
    - cutoff: The cutoff frequency or frequencies.
    - fs: Sampling frequency of the signal.
    - filter_band: Type of filter ('low', 'high', 'bandpass', 'bandstop').
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    for col in columns:
        df[col] = butterworth_filter(df[col].values, order, cutoff, fs, filter_band)
    
    return df

In [13]:
def apply_notch(df, columns, notch_freq=60, fs=1000, Q=30):
    '''
    Apply a notch filter to remove a specific frequency (e.g., powerline noise at 60Hz)
    to the specified columns in the DataFrame.

    Parameters:
    - df: The dataframe containing signal data.
    - columns: List of columns to apply the notch filter to.
    - notch_freq: Frequency to remove with the notch filter (e.g., 60Hz for powerline noise).
    - fs: Sampling frequency of the signal.
    - Q: Quality factor for the notch filter.
    
    Returns:
    - df: The DataFrame with filtered columns.
    '''
    nyquist = 0.5 * fs
    normalized_frequency = notch_freq / nyquist

    b, a = signal.iirnotch(normalized_frequency, Q)

    for col in columns:
        df[col] = signal.filtfilt(b, a, df[col].values)
    
    return df

In [16]:
#  TODO: Linear interpolation for nulls (idk if we have nulls)

# Pre-processing

In [17]:
#  Drop table if any column missing
if len(df.columns) < 43:
    print("yikes! having missing data is quite embarassing")
else:
    print("lets go")

#  Scale data with minimax
columns_non_time = df.columns[1:]
scaler_minmax = MinMaxScaler()
df[columns_non_time] = scaler_minmax.fit_transform(df[columns_non_time])

lets go


In [18]:
#  Resample data

df = resample_data(df, target_fs=100)

#  Apply butterworth filter
df = apply_filter(df, emg_columns)

#  Apply notch filter
df = apply_notch(df, emg_columns)

# Modeling

In [41]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [42]:
def split_dataframe(df):
    time_column = df.columns[0]
    
    sensor_columns = df.columns[1:]

    left_keywords = ['sensor 0', 'sensor 1', 'sensor 2']

    left_sensors = [col for col in sensor_columns if any(keyword in col for keyword in left_keywords)]
    right_sensors = [col for col in sensor_columns if col not in left_sensors]

    left_emg = [col for col in left_sensors if 'EMG' in col]
    right_emg = [col for col in right_sensors if 'EMG' in col]

    left_inputs = [col for col in left_sensors if col not in left_emg]
    right_inputs = [col for col in right_sensors if col not in right_emg]

    return df[left_inputs], df[left_emg], df[right_inputs], df[right_emg]

In [43]:
def create_sliding_windows(features, targets, window_size, target_size, step_size):
    X, y = [], []

    for i in range(0, len(features) - window_size - target_size + 1, step_size):
        X.append(features.iloc[i:i + window_size].values)
        y.append(targets.iloc[i + window_size:i + window_size + target_size].values)

    return np.array(X), np.array(y)

In [45]:

# Start with df with shape (number_of_samples, number_of_columns)

left_sensors, left_emg, right_sensors, right_emg = split_dataframe(df)

#  Small window, step, target for real-time prediction??

window_size = 10
step_size = 1
target_size = 5

#  Sensors and EMG used to predict other leg EMG

left_combined = pd.concat([left_sensors, left_emg], axis=1)
right_combined = pd.concat([right_sensors, right_emg], axis=1)
X_left, y_left = create_sliding_windows(left_combined, right_emg, window_size, target_size, step_size)
X_right, y_right = create_sliding_windows(right_combined, left_emg, window_size, target_size, step_size)
#  Shape = (batch_size, sequence_length, input_size)


#  TODO: determine the real-time delay from prediction to signals transmission sent so that we know how to optimize these parameters. 
#  For example, this delay will affect choice of target size because we might need multiple predictions in advance. 

input_size = 21  #  Number of features per time step (left sensors + left EMG)
output_size = 3   #  Number of output features
target_size = 5   #  Number of time steps in target

hidden_size = 64  #  Number of hidden units
num_layers = 2    #  Number of LSTM layers


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, target_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.target_size = target_size
    
    def forward(self, x):
        out, _ = self.lstm(x)  # Out has shape (batch_size, seq_length, hidden_size)
        out = out[:, -self.target_size:, :]  #  Get the output for the last `target_size` time steps
        out = self.fc(out)  #  Apply the output layer for each time step
        return out

#  Train/test split 
X_left_train, X_left_temp, y_right_train, y_right_temp = train_test_split(X_left, y_right, test_size=0.2, random_state=42)
X_left_val, X_left_test, y_right_val, y_right_test = train_test_split(X_left_temp, y_right_temp, test_size=0.5, random_state=42)

#  Convert to PyTorch tensors
X_left_train_tensor = torch.tensor(X_left_train, dtype=torch.float32)
y_right_train_tensor = torch.tensor(y_right_train, dtype=torch.float32)
X_left_val_tensor = torch.tensor(X_left_val, dtype=torch.float32)
y_right_val_tensor = torch.tensor(y_right_val, dtype=torch.float32)
X_left_test_tensor = torch.tensor(X_left_test, dtype=torch.float32)
y_right_test_tensor = torch.tensor(y_right_test, dtype=torch.float32)

model = LSTMModel(input_size, hidden_size, num_layers, output_size, target_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

patience = 25  #  Number of epochs to wait for improvement
best_loss = float('inf')  #  Initialize the best loss to infinity
patience_counter = 0  #  Counter to track how many epochs we've waited without improvement
best_model_wts = None  #  To store the best model weights

#  Training loop with validation and testing (left leg predict right leg)
num_epochs = 100
for epoch in range(num_epochs):
    model.train() # Training mode
    
    #  Forward pass (training)
    outputs_train = model(X_left_train_tensor)  # Get model predictions
    loss_train = criterion(outputs_train, y_right_train_tensor)  #  Compute loss on training data
    
    #  Backward pass and optimization
    optimizer.zero_grad()  #  Clear previous gradients
    loss_train.backward()  #  Backpropagate gradients
    optimizer.step()  #  Update model parameters
    
    #  Validation
    model.eval()  #  Eval mode
    with torch.no_grad():  #  Disable gradient calculation for validation
        outputs_val = model(X_left_val_tensor)  #  Get model predictions on validation data
        loss_val = criterion(outputs_val, y_right_val_tensor)  #  Compute loss on validation data
    
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {loss_train.item():.6f}, "
          f"Validation Loss: {loss_val.item():.6f}")

    #  Early stopping logic
    if loss_val < best_loss:
        best_loss = loss_val
        patience_counter = 0  #  Reset patience counter if validation loss improved
        best_model_wts = model.state_dict()  #  Save the model weights
    else:
        patience_counter += 1  #  Increment patience counter if no improvement
    
    #  If the patience counter exceeds the threshold, stop training
    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

#  After training is complete, test the model
model.eval()  # Eval mode
with torch.no_grad():  #  Disable gradient calculation for testing
    outputs_test = model(X_left_test_tensor)  #  Get model predictions on test data
    loss_test = criterion(outputs_test, y_right_test_tensor)  #  Compute loss on test data

print(f"Test Loss: {loss_test.item():.6f}")


Epoch [1/100], Train Loss: 0.227929, Validation Loss: 0.208164
Epoch [2/100], Train Loss: 0.208176, Validation Loss: 0.188968
Epoch [3/100], Train Loss: 0.188986, Validation Loss: 0.169776
Epoch [4/100], Train Loss: 0.169798, Validation Loss: 0.150229
Epoch [5/100], Train Loss: 0.150256, Validation Loss: 0.130030
Epoch [6/100], Train Loss: 0.130061, Validation Loss: 0.108961
Epoch [7/100], Train Loss: 0.108994, Validation Loss: 0.086980
Epoch [8/100], Train Loss: 0.087013, Validation Loss: 0.064384
Epoch [9/100], Train Loss: 0.064415, Validation Loss: 0.042058
Epoch [10/100], Train Loss: 0.042084, Validation Loss: 0.021865
Epoch [11/100], Train Loss: 0.021883, Validation Loss: 0.007188
Epoch [12/100], Train Loss: 0.007197, Validation Loss: 0.003064
Epoch [13/100], Train Loss: 0.003061, Validation Loss: 0.012092
Epoch [14/100], Train Loss: 0.012082, Validation Loss: 0.023936
Epoch [15/100], Train Loss: 0.023926, Validation Loss: 0.027310
Epoch [16/100], Train Loss: 0.027301, Validation 

In [46]:
#  Save the model weights
torch.save(model.state_dict(), 'lstm_weights_1.pth')